In [ ]:
import os
import pandas as pd
#estrazione dataset
nomi_folder = os.listdir("dataset_MMSI")
df_dataset= pd.DataFrame()
for nome_file in nomi_folder:
    df = pd.read_csv(os.path.join("dataset_MMSI", nome_file));
    df_dataset = pd.concat([df_dataset, df]);



In [ ]:
df_dataset_original=df_dataset.copy() #nuovo dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_dataset_for_cluster=df_dataset_original.copy()
df_dataset_for_cluster.drop(['MMSI','BaseDateTime','Time','LAT','LON','DistMtr','Status','Heading','COG'], axis=1, inplace=True)
df_dataset_for_cluster.reset_index(drop=True)
scaler.fit(df_dataset_for_cluster)



In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA


# Riduzione della dimensionalità utilizzando PCA
pca = PCA(n_components=0.99)  # Manteniamo il 95% della varianza
df_dataset_for_cluster = pca.fit_transform(df_dataset_for_cluster)


In [ ]:
from sklearn.mixture import GaussianMixture
# Addestramento del modello GMM
n_components = 5  # Definiamo il numero di componenti (cluster)
gmm = GaussianMixture(n_components=n_components)
gmm.fit(df_dataset_for_cluster)
# Predizione dei cluster
clusters = gmm.predict(df_dataset_for_cluster)
del df_dataset_for_cluster
df_dataset_original['state'] = clusters

In [ ]:
#subset = df_dataset_original[df_dataset_original['MMSI'] == 259790000]
subset = df_dataset_original[df_dataset_original['MMSI'] == 636015465]
#subset = df_dataset_original[df_dataset_original['MMSI'] == 636017317]
#subset = df_dataset_original[df_dataset_original['MMSI'] == 367309860]
#subset = df_dataset_original[df_dataset_original['MMSI'] == 367310240]# nave ferma al porto
#subset = df_dataset_original[df_dataset_original['MMSI'] == 219253000]# nave ferma al porto





In [ ]:
import folium
from folium.plugins import HeatMap
from branca.element import Template, MacroElement

# Crea un dizionario di colori per ciascuno stato
color_dict = {0: 'blue', 1: 'green', 2: 'orange', 3: 'red',4:'violet',5:'cyan'}

mapp = folium.Map([21, -158], zoom_start=6, tiles='cartodbpositron')

for i in range(0, len(subset)):
    state = subset.iloc[i]['state']
    color = color_dict.get(state, 'black')  # Utilizza il colore corrispondente allo stato, o 'black' se lo stato non è nel dizionario
    Head = subset.iloc[i]['Heading']  # Angolo COG
    icon_angle = f"""transform: rotate({Head}deg);"""  # Applica la rotazione dell'icona in base all'angolo COG
    folium.Marker(
        location=[subset.iloc[i]['LAT'], subset.iloc[i]['LON']],
        icon=folium.DivIcon(html=f"""<div style="font-size: 12pt; color: {color}; {icon_angle}">➤</div>"""),
        popup=str(state)
    ).add_to(mapp)

# Crea una heatmap
mapp

In [ ]:

subset.drop(['MMSI','BaseDateTime','Time','LAT','LON','DistMtr','Status','Heading','COG'], axis=1, inplace=True)
subset.reset_index(drop=True)
# Aggregazione per gruppo e calcoli dei valori medi delle caratteristiche
# considerando le osservazioni di ogni gruppo

aggdata = subset.groupby('state').mean().reset_index()
aggdata

In [ ]:

aggdata = subset.groupby('state').mean().reset_index()
aggdata